# Downlaod data và giải nén

In [1]:
import gdown

# https://drive.google.com/file/d/1wB67AZv3zh8UUccih-ybDIGnSGq9fLNb/view?usp=drive_link

# Google Drive file ID
file_id = "1wB67AZv3zh8UUccih-ybDIGnSGq9fLNb"
output = "data.zip"

# Tải file từ Google Drive
gdown.download(f"https://drive.google.com/uc?id={file_id}", output, quiet=False)


Downloading...
From (original): https://drive.google.com/uc?id=1wB67AZv3zh8UUccih-ybDIGnSGq9fLNb
From (redirected): https://drive.google.com/uc?id=1wB67AZv3zh8UUccih-ybDIGnSGq9fLNb&confirm=t&uuid=1a0ac773-b789-4e75-91bb-deccf8485795
To: /content/data.zip
100%|██████████| 642M/642M [00:13<00:00, 47.2MB/s]


'data.zip'

In [2]:
import zipfile

with zipfile.ZipFile("data.zip", 'r') as zip_ref:
    zip_ref.extractall("data")  # Thư mục đích

print("Đã giải nén xong!")


Đã giải nén xong!


In [3]:
import time
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from skimage.transform import resize
from skimage import feature
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

# Read and extract data

- Các list cần tạo: label_train, img_train, label_test, img_test, label_meta, img_meta

- Các trường trog file csv: Width, Height, Roi.X1, Roi.Y1, Roi.X2, Roi.Y2, ClassId, Path


## 1. Đọc dữ liệu tập train

In [4]:
# Đường dẫn đến thư mục train
data_dir = '/content/data'
csv_train_path = os.path.join(data_dir, 'Train.csv')

# Đọc file CSV
df = pd.read_csv(csv_train_path)

# Khởi tạo danh sách lưu ảnh và nhãn
img_train = []
label_train = []

# Duyệt từng dòng trong dataframe
for index, row in df.iterrows():
    # Lấy thông tin từ các cột
    path_img = os.path.join(data_dir, row['Path'])  # Nối đường dẫn đầy đủ
    x1, y1, x2, y2 = int(row['Roi.X1']), int(row['Roi.Y1']), int(row['Roi.X2']), int(row['Roi.Y2'])
    class_id = row['ClassId']

    # Đọc ảnh
    img = cv2.imread(path_img)

    # Kiểm tra ảnh có tồn tại và hợp lệ không
    if img is not None:
        # Cắt ảnh theo ROI
        roi = img[y1:y2, x1:x2]

        # Lưu vào danh sách
        img_train.append(roi)
        label_train.append(class_id)
    else:
        print(f"Lỗi: Không đọc được ảnh {path_img}")


In [5]:
print('Number of objects: ', len(img_train))
print('Class names: ', list(set(label_train)))

Number of objects:  39209
Class names:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42]


## 2. Đọc dữ liệu tập test

In [6]:
csv_test_path = os.path.join(data_dir, 'Test.csv')

# Đọc file CSV
df = pd.read_csv(csv_test_path)

# Khởi tạo danh sách lưu ảnh và nhãn
img_test = []
label_test = []

# Duyệt từng dòng trong dataframe
for index, row in df.iterrows():
    # Lấy thông tin từ các cột
    path_img = os.path.join(data_dir, row['Path'])  # Nối đường dẫn đầy đủ
    x1, y1, x2, y2 = int(row['Roi.X1']), int(row['Roi.Y1']), int(row['Roi.X2']), int(row['Roi.Y2'])
    class_id = row['ClassId']

    # Đọc ảnh
    img = cv2.imread(path_img)

    # Kiểm tra ảnh có tồn tại và hợp lệ không
    if img is not None:
        # Cắt ảnh theo ROI
        roi = img[y1:y2, x1:x2]

        # Lưu vào danh sách
        img_test.append(roi)
        label_test.append(class_id)
    else:
        print(f"Lỗi: Không đọc được ảnh {path_img}")


In [7]:
print('Number of objects: ', len(img_test))
print('Class names: ', list(set(label_test)))

Number of objects:  12630
Class names:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42]


# Preprocess image

In [8]:
def preprocess_img(img):
    if len(img.shape) > 2:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = img.astype(np.float32)

    resized_img = resize(
        img,
        output_shape=(32, 32),
        anti_aliasing=True
    )

    hog_feature = feature.hog(
        resized_img,
        orientations=9,
        pixels_per_cell=(8, 8),
        cells_per_block=(2, 2),
        transform_sqrt=True,
        block_norm="L2",
        feature_vector=True
    )

    return hog_feature

In [9]:
img_features_train_lst = []
for img in img_train:
    hog_feature = preprocess_img(img)
    img_features_train_lst.append(hog_feature)

img_features_train = np.array(img_features_train_lst)
print('X shape:')
print(img_features_train.shape)

print(len(label_train))

X shape:
(39209, 324)
39209


In [10]:
img_features_test_lst = []
for img in img_test:
    hog_feature = preprocess_img(img)
    img_features_test_lst.append(hog_feature)

img_features_test = np.array(img_features_test_lst)
print('X shape:')
print(img_features_test.shape)

print(len(label_test))

X shape:
(12630, 324)
12630


# Chia tập train, val

In [11]:
random_state = 0
test_size = 0.3
is_shuffle = True

X_train, X_val, y_train, y_val = train_test_split(
    img_features_train, label_train,
    test_size=test_size,
    random_state=random_state,
    shuffle=is_shuffle
)

# Normalization

In [12]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(img_features_test)

In [13]:
print(X_train.shape)
print(len(y_train))

(27446, 324)
27446


In [14]:
print(X_val.shape)
print(len(y_val))

(11763, 324)
11763


In [15]:
y_test = label_test

print(X_test.shape)
print(len(y_test))

(12630, 324)
12630


#Training

In [16]:
clf = SVC(
    C=0.75,
    kernel='rbf',
    gamma='scale'
)
clf.fit(X_train, y_train)

SVC(C=0.75)

# Evaluation

## Tập val

In [17]:
y_pred_val = clf.predict(X_val)
score_val = accuracy_score(y_pred_val, y_val)

print('Evaluation results on val set')
print('Accuracy: ', score_val)

# print(classification_report(y_val, y_pred_val))


Evaluation results on val set
Accuracy:  0.9669302048797076
              precision    recall  f1-score   support

           0       1.00      0.94      0.97        50
           1       0.90      0.90      0.90       679
           2       0.93      0.92      0.92       655
           3       0.98      0.96      0.97       418
           4       0.99      0.99      0.99       625
           5       0.84      0.86      0.85       548
           6       1.00      1.00      1.00       139
           7       0.96      0.94      0.95       433
           8       0.92      0.93      0.92       405
           9       1.00      0.99      1.00       418
          10       1.00      0.99      0.99       607
          11       0.92      0.97      0.95       402
          12       0.97      1.00      0.98       628
          13       1.00      1.00      1.00       673
          14       1.00      0.99      0.99       243
          15       0.99      0.99      0.99       186
          16       1.

## Tập test

In [18]:
y_pred_test = clf.predict(X_test)
score_test = accuracy_score(y_pred_test, y_test)

print('Evaluation results on test set')
print('Accuracy: ', score_test)

# print(classification_report(y_test, y_pred_test))

Evaluation results on test set
Accuracy:  0.8923198733174981
              precision    recall  f1-score   support

           0       1.00      0.52      0.68        60
           1       0.76      0.80      0.78       720
           2       0.86      0.91      0.89       750
           3       0.92      0.84      0.88       450
           4       0.97      0.95      0.96       660
           5       0.73      0.80      0.76       630
           6       0.96      0.70      0.81       150
           7       0.93      0.87      0.90       450
           8       0.84      0.84      0.84       450
           9       0.91      1.00      0.95       480
          10       0.99      0.99      0.99       660
          11       0.72      0.85      0.78       420
          12       0.91      1.00      0.95       690
          13       1.00      1.00      1.00       720
          14       1.00      0.94      0.97       270
          15       1.00      0.99      0.99       210
          16       1

## Kernel: sigmoid

In [ ]:
clf_sigmoid = SVC(
    kernel='sigmoid',
    C=0.5
)
clf_sigmoid.fit(X_train, y_train)

In [ ]:
y_pred_sigmoid = clf_sigmoid.predict(X_test)
score_sigmoid = accuracy_score(y_pred_sigmoid, y_test)

print('Evaluation results on test set')
print('Accuracy: ', score_sigmoid)
# print(classification_report(y_test, y_pred_sigmoid))

## Kernel: Poly

In [ ]:
clf_poly = SVC(
    kernel='poly',
    C=0.5
)
clf_poly.fit(X_train, y_train)

In [ ]:
y_pred_poly = clf_poly.predict(X_test)
score_poly = accuracy_score(y_pred_poly, y_test)

print('Evaluation results on test set')
print('Accuracy: ', score_poly)
# print(classification_report(y_test, y_pred_poly))

## Kernel: Linear

In [23]:
clf_linear = SVC(
    kernel='linear',
    # C=0.5
)
clf_linear.fit(X_train, y_train)

SVC(kernel='linear')

In [24]:
y_pred_linear_val = clf_linear.predict(X_val)
score_linear_val = accuracy_score(y_pred_linear_val, y_val)

print('Evaluation results on val set')
print('Accuracy: ', score_linear_val)
print("Number of support vectors:", len(clf_linear.support_))

Evaluation results on val set
Accuracy:  0.9529881832865765
Number of support vectors: 8959


In [25]:
y_pred_linear = clf_linear.predict(X_test)
score_linear = accuracy_score(y_pred_linear, y_test)

print('Evaluation results on test set')
print('Accuracy: ', score_linear)
print("Number of support vectors:", len(clf_linear.support_))

# print(classification_report(y_test, y_pred_linear))

Evaluation results on test set
Accuracy:  0.8794932699920823
Number of support vectors: 8959


# Tải model

In [24]:
import joblib

joblib.dump(clf, 'svm_model.pkl')


['svm_model.pkl']

In [25]:
from google.colab import files
files.download('svm_model.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Check model đã train

In [26]:
!pip install -U gdown
import gdown

# https://drive.google.com/file/d/1eIL5FlvbWTKe-_hMa1HoyzSvQ9vkDjQP/view?usp=sharing
file_id = "1eIL5FlvbWTKe-_hMa1HoyzSvQ9vkDjQP"
url = f"https://drive.google.com/uc?id={file_id}"

output = 'svm_model.pkl'
gdown.download(url, output, quiet=False)


Downloading...
From: https://drive.google.com/uc?id=1eIL5FlvbWTKe-_hMa1HoyzSvQ9vkDjQP
To: /content/svm_model.pkl
100%|██████████| 43.7M/43.7M [00:00<00:00, 126MB/s]


'svm_model.pkl'

In [27]:
import joblib

model = joblib.load('svm_model.pkl')


In [28]:
y_pred1 = model.predict(X_test)

print(classification_report(y_test, y_pred1))


              precision    recall  f1-score   support

           0       1.00      0.52      0.68        60
           1       0.76      0.80      0.78       720
           2       0.86      0.91      0.89       750
           3       0.92      0.84      0.88       450
           4       0.97      0.95      0.96       660
           5       0.73      0.80      0.76       630
           6       0.96      0.70      0.81       150
           7       0.93      0.87      0.90       450
           8       0.84      0.84      0.84       450
           9       0.91      1.00      0.95       480
          10       0.99      0.99      0.99       660
          11       0.72      0.85      0.78       420
          12       0.91      1.00      0.95       690
          13       1.00      1.00      1.00       720
          14       1.00      0.94      0.97       270
          15       1.00      0.99      0.99       210
          16       1.00      0.98      0.99       150
          17       0.99    

In [29]:
print(confusion_matrix(y_test, y_pred1))

[[ 31  20   0 ...   0   0   0]
 [  0 578  47 ...   0   0   0]
 [  0  37 684 ...   2   0   0]
 ...
 [  0   5   1 ...  78   0   0]
 [  0   1   0 ...   0  23   3]
 [  0   2   1 ...   0  14  64]]
